In [ ]:
%load_ext autoreload
%autoreload 2



%autoreload 2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.spatial.distance import pdist, squareform
from scipy.special import expit
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering  # Add this import
import numpy as np
import pandas as pd
import sys
sys.path.append('/Users/sarahurbut/aladynoulli2/pyScripts')
sys.path.append('/Users/sarahurbut/aladynoulli2/pyScripts/new_oct_revision')
from utils import *
import sys
import os
import gc


def subset_data(Y, E, G, start_index, end_index):
    """Subset data based on indices."""
    indices = list(range(start_index, end_index))
    Y_subset = Y[indices]
    E_subset = E[indices]
    G_subset = G[indices]
    return Y_subset, E_subset, G_subset, indices

def load_model_essentials(base_path='/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/data_for_running/'):
    """
    Load all essential components
    """
    print("Loading components...")
    
    # Load large matrices
    Y = torch.load(base_path + 'Y_tensor.pt')
    E = torch.load(base_path + 'E_enrollment_full.pt')
    G = torch.load(base_path + 'G_matrix.pt')
    
    # Load other components
    essentials = torch.load(base_path + 'model_essentials.pt')
    
    print("Loaded all components successfully!")
    
    return Y, E, G, essentials

# Load and initialize model:
Y, E, G, essentials = load_model_essentials()

    
    # Path to your total fit model
from clust_huge_amp_fixedPhi import *
total_fit_path = '/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/enrollment_model_W0.0001_fulldata_sexspecific.pt'
total_checkpoint = torch.load(total_fit_path, map_location='cpu')
phi_total = total_checkpoint['model_state_dict']['phi'].cpu().numpy()  # shape: (K, D, T)
psi_total = total_checkpoint['model_state_dict']['psi'].cpu().numpy()  # shape: (K, D, T)


Loading components...


/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_65939/1627225843.py:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Y = torch.load(base_path + 'Y_tensor.pt')
/v

Loaded all components successfully!


/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_65939/1627225843.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  total_checkpoint = torch.load(total_fit_path

In [2]:

# Load the RDS file

import pandas as pd

fh_processed=pd.read_csv('/Users/sarahurbut/Library/Cloudstorage/Dropbox-Personal/baselinagefamh_full_withpcs.csv')
len(fh_processed)
disease_names=essentials['disease_names']
prevalence_logit=essentials['prevalence_t']


In [3]:
total_fit_path = '/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/enrollment_model_W0.0001_fulldata_sexspecific.pt'
total_checkpoint = torch.load(total_fit_path, map_location='cpu')
phi = total_checkpoint['model_state_dict']['phi'].cpu().numpy()  # shape: (K, D, T)
psi_total = total_checkpoint['model_state_dict']['psi'].cpu().numpy()

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_65939/3965418255.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  total_checkpoint = torch.load(total_fit_path,

In [4]:
fh_processed.head()

,identifier,age,sex,f.22009.0.1,f.22009.0.2,f.22009.0.3,f.22009.0.4,f.22009.0.5,f.22009.0.6,f.22009.0.7,f.22009.0.8,f.22009.0.9,f.22009.0.10
0,1000015,69,1,-12.02340,3.25100,1.224590,4.776840,2.31802,2.445040,-3.067290,-1.08792,0.550964,1.201910
1,1000023,44,1,-14.07040,2.83809,-1.709870,-0.782455,-7.93571,-2.176480,-0.688286,-2.59305,4.890120,5.056270
2,1000037,69,0,-15.00630,4.03881,-1.711670,6.200160,-2.77040,0.809991,1.012190,3.03839,5.836320,-1.955720
3,1000042,66,1,-12.08720,6.29019,-1.486040,-3.426860,-5.35913,-0.146506,2.788340,-4.25511,-6.282060,-0.862379
4,1000059,54,0,-9.58886,3.77243,0.274298,-1.357730,1.91795,-1.627780,-0.191782,-1.28348,2.726250,1.320520


In [ ]:
import torch
import numpy as np
import pandas as pd
import gc
import cProfile
import pstats
from pstats import SortKey
from clust_huge_amp_fixedPhi import *

# ============================================================================
# CONFIGURATION
# ============================================================================
# Define all batches (adjust end point to your actual dataset size)
batches = [
    (0, 10000)]
    '''
    (10000, 20000)],

    (20000, 30000),
    (30000, 40000),
    (40000, 50000),
    (50000, 60000),
    (60000, 70000),
    (70000, 80000),
    (80000, 90000),
    (90000, 100000),
    (100000, 110000),
    (110000, 120000),
    (120000, 130000),
    (130000, 140000),
    (140000, 150000),
    (150000, 160000),
    (160000, 170000),
    (170000, 180000),
    (180000, 190000),
    (190000, 200000),
    (200000, 210000),
    (210000, 220000),
    (220000, 230000),
    (230000, 240000),
    (240000, 250000),
    (250000, 260000),
    (260000, 270000),
    (270000, 280000),
    (280000, 290000),
    (290000, 300000),
    (300000, 310000),
    (310000, 320000),
    (320000, 330000),
    (330000, 340000),
    (340000, 350000),
    (350000, 360000),
    (360000, 370000),
    (370000, 380000),
    (380000, 390000),
    (390000, 400000),
    # Adjust to your actual size
]
'''
output_dir = "/Users/sarahurbut/Library/CloudStorage/Dropbox/enrollment_predictions_fixedphi_withpcs/"

# Create output directory if it doesn't exist
import os
os.makedirs(output_dir, exist_ok=True)

print(f"Will process {len(batches)} batches")
print(f"Output directory: {output_dir}")

# ============================================================================
# PROCESS EACH BATCH
# ============================================================================
for batch_idx, (start, stop) in enumerate(batches):
    print(f"\n{'='*80}")
    print(f"BATCH {batch_idx+1}/{len(batches)}: Processing patients {start} to {stop}")
    print(f"{'='*80}")
    
    try:
        # Set random seeds for reproducibility
        torch.manual_seed(42)
        np.random.seed(42)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(42)
            torch.backends.cudnn.deterministic = True
        
        # Subset the data
        print(f"Subsetting data...")
        Y_batch, E_batch, G_batch, indices = subset_data(Y, E, G, 
                                                          start_index=start, 
                                                          end_index=stop)
        
        # Get demographics and add sex
        pce_df_subset = fh_processed.iloc[start:stop].reset_index(drop=True)
        sex = pce_df_subset['sex'].values
        pc_columns = ['f.22009.0.1', 'f.22009.0.2', 'f.22009.0.3', 'f.22009.0.4', 'f.22009.0.5',
              'f.22009.0.6', 'f.22009.0.7', 'f.22009.0.8', 'f.22009.0.9', 'f.22009.0.10']
        pcs = pce_df_subset[pc_columns].values

# Stack: G_batch + sex + 10 PCs
        G_with_sex_pcs = np.column_stack([G_batch, sex, pcs])
        
        print(f"Data shapes: Y={Y_batch.shape}, E={E_batch.shape}, G={G_with_sex.shape}")
        
        # Initialize model with FIXED phi and psi
        print(f"Initializing model with fixed phi/psi...")
        model = AladynSurvivalFixedPhi(
            N=Y_batch.shape[0],
            D=Y_batch.shape[1],
            T=Y_batch.shape[2],
            K=20,
            P=G_with_sex.shape[1],
            G=G_with_sex,
            Y=Y_batch,
            R=0,
            W=0.0001,
            prevalence_t=essentials['prevalence_t'],
            init_sd_scaler=1e-1,
            genetic_scale=1,
            pretrained_phi=phi_total,
            pretrained_psi=psi_total,
            signature_references=signature_refs,
            healthy_reference=True,
            disease_names=essentials['disease_names']
        )
        
        # Verify phi and psi are fixed
        if np.allclose(model.phi.cpu().numpy(), phi_total):
            print("✓ phi matches phi_total!")
        else:
            print("✗ WARNING: phi does NOT match phi_total!")
            
        if np.allclose(model.psi.cpu().numpy(), psi_total):
            print("✓ psi matches psi_total!")
        else:
            print("✗ WARNING: psi does NOT match psi_total!")
        
        # Train model (only lambda is being estimated)
        print(f"Training model...")
        profiler = cProfile.Profile()
        profiler.enable()
        
        history = model.fit(
            E_batch, 
            num_epochs=200, 
            learning_rate=1e-1, 
            lambda_reg=1e-2
        )
        
        profiler.disable()
        stats = pstats.Stats(profiler).sort_stats(SortKey.CUMULATIVE)
        stats.print_stats(10)
        
        # Generate and save predictions
        print(f"Generating predictions...")
        with torch.no_grad():
            pi, _, _ = model.forward()
            
            # Save predictions
            pi_filename = os.path.join(output_dir, 
                                       f"pi_enroll_fixedphi_sex_{start}_{stop}.pt")
            torch.save(pi, pi_filename)
            print(f"✓ Saved predictions to {pi_filename}")
            
            # Save model state (optional, for debugging)
            model_filename = os.path.join(output_dir, 
                                         f"model_enroll_fixedphi_sex_{start}_{stop}.pt")
            torch.save({
                'model_state_dict': model.state_dict(),
                'E': E_batch,
                'prevalence_t': model.prevalence_t,
                'logit_prevalence_t': model.logit_prev_t,
                'start_index': start,
                'end_index': stop,
                #'history': history
            }, model_filename)
            print(f"✓ Saved model to {model_filename}")
        
        # Clean up to free memory
        print(f"Cleaning up memory...")
        del pi, model, Y_batch, E_batch, G_batch, G_with_sex, pce_df_subset
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        
        print(f"✓ Batch {batch_idx+1}/{len(batches)} complete!")
        
    except Exception as e:
        print(f"✗ ERROR in batch {start}-{stop}: {e}")
        import traceback
        traceback.print_exc()
        continue

print(f"\n{'='*80}")
print(f"ALL BATCHES COMPLETE!")
print(f"{'='*80}")

# ============================================================================
# CONCATENATE ALL PREDICTIONS INTO ONE FILE
# ============================================================================
print(f"\nConcatenating all predictions into single file...")
pi_batches = []
successful_batches = []

for start, stop in batches:
    pi_filename = os.path.join(output_dir, f"pi_enroll_fixedphi_sex_{start}_{stop}.pt")
    try:
        pi_batch = torch.load(pi_filename)
        pi_batches.append(pi_batch)
        successful_batches.append((start, stop))
        print(f"✓ Loaded {pi_filename}, shape: {pi_batch.shape}")
    except Exception as e:
        print(f"✗ Could not load {pi_filename}: {e}")

if pi_batches:
    print(f"\nConcatenating {len(pi_batches)} batches...")
    pi_full = torch.cat(pi_batches, dim=0)
    print(f"Final shape: {pi_full.shape}")
    
    # Save combined file
    full_filename = os.path.join(output_dir, "pi_enroll_fixedphi_sex_FULL.pt")
    torch.save(pi_full, full_filename)
    print(f"✓ Saved combined predictions to {full_filename}")
    
    # Save batch info
    batch_info = {
        'batches': successful_batches,
        'total_patients': pi_full.shape[0],
        'n_diseases': pi_full.shape[1],
        'n_timepoints': pi_full.shape[2]
    }
    info_filename = os.path.join(output_dir, "batch_info.pt")
    torch.save(batch_info, info_filename)
    print(f"✓ Saved batch info to {info_filename}")
else:
    print("✗ No successful batches to concatenate!")

print(f"\n{'='*80}")
print(f"DONE! Ready for washout analysis.")
print(f"{'='*80}")

In [ ]:
history=tl['history']

In [ ]:
import torch
import os

dir_path = "/Users/sarahurbut/Library/CloudStorage/Dropbox/enrollment_predictions_fixedphi/"

# Get all model files
model_files = [f for f in os.listdir(dir_path) 
               if f.startswith('model_enroll_fixedphi_sex_') and f.endswith('.pt')]

print(f"Found {len(model_files)} model files")
print("Removing history from all checkpoints...\n")

for filename in model_files:
    filepath = os.path.join(dir_path, filename)
    
    # Get original size
    original_size = os.path.getsize(filepath) / 1e9
    print(f"Cleaning {filename} ({original_size:.2f} GB)...")
    
    # Load checkpoint
    checkpoint = torch.load(filepath)
    
    # Remove history and clone tensors
    clean_checkpoint = {
        'model_state_dict': {k: v.clone() if isinstance(v, torch.Tensor) else v 
                            for k, v in checkpoint['model_state_dict'].items()},
        'E': checkpoint['E'].clone(),
        'prevalence_t': checkpoint['prevalence_t'].clone(),
        'logit_prevalence_t': checkpoint['logit_prevalence_t'].clone(),
        'start_index': checkpoint['start_index'],
        'end_index': checkpoint['end_index']
        # NO HISTORY!
    }
    
    # Save cleaned version
    torch.save(clean_checkpoint, filepath)
    
    new_size = os.path.getsize(filepath) / 1e9
    saved = original_size - new_size
    
    print(f"✓ {original_size:.2f} GB → {new_size:.3f} GB (saved {saved:.2f} GB)\n")
    
    # Clean up
    del checkpoint, clean_checkpoint

print("✅ All files cleaned! History removed.")

In [ ]:
# Load the RDS file
readRDS = robjects.r['readRDS']
pce_data = readRDS('/Users/sarahurbut/Library/Cloudstorage/Dropbox-Personal/pce_df_prevent.rds')
pce_df = pandas2ri.rpy2py(pce_data)  # Convert to pandas DataFrame


In [ ]:
# Test if import works
try:
    from evaluatetdccode import evaluate_major_diseases_wsex_with_bootstrap_dynamic_1year_different_start_end_numeric_sex
    print("✅ Import successful")
except ImportError as e:
    print(f"❌ Import failed: {e}")



In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from evaluatetdccode import *

In [ ]:
import torch
import pandas as pd
from evaluatetdccode import evaluate_major_diseases_wsex_with_bootstrap_dynamic_1year_different_start_end_numeric_sex

# Load the full data once
Y, E, G, essentials = load_model_essentials()
fh_processed = pd.read_csv('/Users/sarahurbut/Library/Cloudstorage/Dropbox-Personal/baselinagefamh.csv')

# Define batches (same as training)
batches = [
    (0, 10000)#, (10000, 20000), (20000, 30000), 
    # ... add all your batches
]

# Storage for results
washout_results = {
    '0yr': {},  # No washout
    '1yr': {},  # 1-year washout  
    '2yr': {}   # 2-year washout
}

# Run washout analysis on each batch
for start, stop in batches:
    print(f"\n=== Processing batch {start}-{stop} ===")
    
    # Load batch predictions
    pi_filename = f"/Users/sarahurbut/Library/CloudStorage/Dropbox/enrollment_predictions_fixedphi/pi_enroll_fixedphi_sex_{start}_{stop}.pt"
    pi_batch = torch.load(pi_filename)
    
    # Subset other data to match
    Y_batch = Y[start:stop]
    E_batch = E[start:stop] 
    pce_df_batch = fh_processed.iloc[start:stop].reset_index(drop=True)
    
    # Run washout analysis for this batch
    for washout_name, offset in [('0yr', 0), ('1yr', 1), ('2yr', 2)]:
        print(f"  Running {washout_name} washout...")
        
        results = evaluate_major_diseases_wsex_with_bootstrap_dynamic_1year_different_start_end_numeric_sex(
            pi=pi_batch,
            Y_100k=Y_batch,
            E_100k=E_batch,
            disease_names=essentials['disease_names'],
            pce_df=pce_df_batch,
            n_bootstraps=50,  # Fewer bootstraps per batch
            follow_up_duration_years=1,
            start_offset=offset
        )
        
        # Store results
        for disease, metrics in results.items():
            if disease not in washout_results[washout_name]:
                washout_results[washout_name][disease] = {
                    'aucs': [], 'cis': [], 'events': [], 'rates': []
                }
            
            washout_results[washout_name][disease]['aucs'].append(metrics['auc'])
            washout_results[washout_name][disease]['cis'].append((metrics['ci_lower'], metrics['ci_upper']))
            washout_results[washout_name][disease]['events'].append(metrics['n_events'])
            washout_results[washout_name][disease]['rates'].append(metrics['event_rate'])
    
    # Clean up memory
    del pi_batch, Y_batch, E_batch, pce_df_batch
    torch.cuda.empty_cache() if torch.cuda.is_available() else None

# Aggregate results across batches
print("\n=== AGGREGATED WASHOUT RESULTS ===")
for washout_name, diseases in washout_results.items():
    print(f"\n{washout_name.upper()} WASHOUT:")
    for disease, metrics in diseases.items():
        aucs = [a for a in metrics['aucs'] if not pd.isna(a)]
        if aucs:
            mean_auc = np.mean(aucs)
            print(f"  {disease}: {mean_auc:.3f} (from {len(aucs)} batches)")

In [ ]:
from plot_washout_results import *